# Initial PyBigDFT tour - Basic functionalities
we start by presenting a broad overview of using the PyBigDFT library to drive BigDFT calculations using Python. In this lesson we will go quickly, returning in the future lessons to go into detail about each part.

For this tutorial, we have to ensure that at least the bigdft client is installed.

In [ ]:
# in the google colab session this can be done with the following command:
!wget https://gitlab.com/luigigenovese/bigdft-school/-/raw/main/packaging/install.py &> /dev/null
import install
install.client(locally=True)

As well as the `py3dmol` package.

In [ ]:
install.packages('py3Dmol')

## System Manipulation
Here we define a system which is composed of two fragments: H2 and Helium.

In [ ]:
from BigDFT.Systems import System
from BigDFT.Fragments import Fragment
from BigDFT.Atoms import Atom

In [ ]:
# Create Three Atoms
at1 = Atom({"H": [0, 0, 0], "units": "bohr"})
at2 = Atom({"H": [0, 0, 1.4], "units": "bohr"})
at3 = Atom({"He": [10, 0, 0], "units": "bohr"})
# Construct a System from Two Fragments (H2, He)
sys = System()
sys["H2:1"] = Fragment([at1, at2])
sys["He:2"] = Fragment([at3])

# Iterate Over The System to inspect the atoms positions
for at in sys.get_atoms():
    print(at.sym, at.get_position())
# or alternatively, two-level iteration (fragment first)
for fragid, frag in sys.items():
    print(fragid)
    for at in frag:
        print(' ', at.sym, at.get_position())

In [ ]:
# this is what we have created
_ = sys.display()

This is a very simple initial example. We will inspect in more details the constructors for a BigDFT system in the System-Generation notebook.

## Calculation
Perform a calculation on the created system using a wavelet grid spacing of $0.4$ bohr (good for the large majority of the systems) and the PBE functional. A logfile is generated from which we can access the computed properties. This logfile has built in properties and can be accessed like a dictionary.

In [ ]:
from BigDFT.Inputfiles import Inputfile
inp = Inputfile()
inp.set_hgrid(0.4)
inp.set_xc("PBE")
# input file is treated like a nested dictionary
inp["perf"] = {"calculate_forces": False}

In [ ]:
from BigDFT.Calculators import SystemCalculator
calc = SystemCalculator(skip=True, verbose=False)

In [ ]:
log = calc.run(sys=sys, input=inp, name="quick", run_dir="scratch")

In [ ]:
#some basic info
print(log.energy)
print(list(log.evals))
print(log.log["Memory Consumption Report"]
             ["Memory occupation"])

We have "executed" (the calculation result is provided with the school material here) a very simple calculation with two Kohn-Sham orbitals. The description of the possible input parameters and how to trigger larger calculations (even on remote computers) will be provided in the forthcoming lessons.

## Calculation of Periodic Systems
We setup a BCC unit cell of iron and perform the calculation using a 2x2x2 k-point grid with a Monkhorst-Pack grid.

In [ ]:
from BigDFT.UnitCells import UnitCell
# one single periodic atom
pat = Atom({"Fe": [0, 0, 0], "units": "angstroem"})
psys = System({"CEL:0": Fragment([pat])})
psys.cell = UnitCell([2.867, 2.867, 2.867], units="angstroem")

In [ ]:
_ = psys.display()

In [ ]:
# very small grid spacing, just for comparison
inp = Inputfile()
inp.set_hgrid(0.3)
inp.set_xc("LDA") # can be omitted as this is the default
inp["kpt"] = {"method": "mpgrid", "ngkpt": [2, 2, 2]}

In [ ]:
log = calc.run(sys=psys, input=inp, name="psys", run_dir="scratch")

In [ ]:
_ = log.get_dos().plot()

After this first tour, we can explore further those functionalities with the following tutorials.

In [ ]:
#this is helpful to preserve the py3dmol installation
install.close_drive()